In [19]:
import drjit as dr
import numpy as np
import matplotlib.pyplot as plt

from tmm import coh_tmm

In [20]:
from drjit.cuda import (
    Float,
    UInt32,
    Int32,
    Array4f,
    Array3f,
    Array2f,
    TensorXf,
    Complex2f,
    Matrix2f,
    Matrix4f,
    Loop,
)

In [21]:
from drjit.cuda import Matrix4f

x = Matrix4f(np.array([[1, 3, 3, 3], [1, 2, 2, 2], [1, 2, 2, 2], [1, 2, 2, 2]]))

In [22]:
# list of layer thicknesses in nm
d_list = [np.inf, 5, 30, np.inf]
# list of refractive indices
n_list = [1.517, 3.719 + 4.362j, 0.130 + 3.162j, 1]
# wavelength in nm
lam_vac = 633
# list of angles to plot
theta_list = np.linspace(np.deg2rad(30), np.deg2rad(60), num=300)
# initialize lists of y-values to plot
Rp = []
for theta in theta_list:
    Rp.append(coh_tmm("p", n_list, d_list, theta, lam_vac)["R"])
# plt.figure()
# plt.plot(np.rad2deg(theta_list), Rp, "blue")
# plt.xlabel("theta (degree)")
# plt.ylabel("Fraction reflected")
# plt.xlim(30, 60)
# plt.ylim(0, 1)
# plt.title(
#     "Reflection of p-polarized light with Surface Plasmon Resonance\n"
#     "Compare with http://doi.org/10.2320/matertrans.M2010003 Fig 6a"
# )

In [24]:
# Represent a complex-valued matrix into real-valued matrix
# https://math.stackexchange.com/a/2012242

def get_rows(z11, z12, z21, z22):
    return (
        Array4f(z11.real, -z11.imag, z12.real, -z12.imag),
        Array4f(z11.imag, z11.real, z12.imag, z12.real),
        Array4f(z21.real, -z21.imag, z22.real, -z22.imag),
        Array4f(z21.imag, z21.real, z22.imag, z22.real)
    )

def get_big_matrix(row0, row1, row2, row3):
    return dr.transpose(Matrix4f(row0, row1, row2, row3))

def get_00(m):
    return Complex2f(m[0, 0], m[1, 0])

def get_10(m):
    return Complex2f(m[2, 0], m[3, 0])

z11 = 1 + 5j
z12 = 2 + 6j
z21 = 3 + 7j
z22 = 4 + 8j
a = np.array(
    [
        [z11, z12],
        [z21, z22],
    ]
)

z11 = Complex2f(np.array([z11]))
z12 = Complex2f(np.array([z12]))
z21 = Complex2f(np.array([z21]))
z22 = Complex2f(np.array([z22]))
A = get_big_matrix(*get_rows(z11, z12, z21, z22))

z11 = 5 + 1j
z12 = 6 + 2j
z21 = 7 + 3j
z22 = 8 + 4j
b = np.array(
    [
        [z11, z12],
        [z21, z22],
    ]
)

z11 = Complex2f(np.array([z11]))
z12 = Complex2f(np.array([z12]))
z21 = Complex2f(np.array([z21]))
z22 = Complex2f(np.array([z22]))
B = get_big_matrix(*get_rows(z11, z12, z21, z22))

t = 1 / (2.5 + 3.0j)
mulab = t * a @ b

t = 1 / Complex2f(np.array([2.5 + 3.0j]))
T = get_big_matrix(*get_rows(t, 0, 0, t))
mulAB = T @ A @ B

print("mulab")
print(mulab)
print("")

print("mulAB")
print(mulAB)

# Since in tmm we have Mtilde[1, 0] / Mtilde[0, 0]
# that means getting:
# z11.real + iz11.imag (Mtilde[0, 0])
# z21.real + iz21.imag (Mtilde[1, 0])
print(mulab[0, 0], mulab[1, 0])
print(get_00(mulAB), get_10(mulAB))

mulab
[[13.90163934+12.91803279j 15.3442623 +16.78688525j]
 [22.81967213+15.01639344j 25.83606557+20.19672131j]]

mulAB
[[[13.901640892028809, -12.918034553527832, 15.34426212310791, -16.78688621520996],
  [12.918034553527832, 13.901639938354492, 16.78688621520996, 15.344263076782227],
  [22.819673538208008, -15.01639461517334, 25.83606719970703, -20.19672203063965],
  [15.01639461517334, 22.819673538208008, 20.19672393798828, 25.83606719970703]]]
(13.901639344262291+12.918032786885245j) (22.819672131147534+15.016393442622949j)
[[13.901640892028809 + 12.918034553527832i]] [[22.819673538208008 + 15.01639461517334i]]


In [ ]:
def is_forward_angle(n, theta):
    ncostheta = n * dr.cos(theta)

    m = dr.abs(ncostheta.imag) > 100 * dr.epsilon(Float)
    answer = dr.select(m, ncostheta.imag > 0, ncostheta.real > 0)

    return answer

def list_snell(n_list, th_0):
    n0_ = n_list[0] * dr.sin(th_0) / n_list

    angles = dr.asin(n0_)

    indices = UInt32([0, dr.width(n_list) - 1])
    n_result = dr.gather(Complex2f, n_list, indices)
    angles_result = dr.gather(Complex2f, angles, indices)

    # The first and last entry need to be the forward angle (the intermediate
    # layers don't matter, see https://arxiv.org/abs/1603.02720 Section 5)
    not_is_forward_angle = ~is_forward_angle(n_result, angles_result)
    angles_result = dr.select(not_is_forward_angle, dr.pi - angles_result, angles_result)
    dr.scatter(angles, angles_result, indices)

    return angles

def interface_t(polarization, n_i, n_f, th_i, th_f):
    if polarization == "s":
        return 2.0 * n_i * dr.cos(th_i) / (n_i * dr.cos(th_i) + n_f * dr.cos(th_f))
    elif polarization == "p":
        return 2.0 * n_i * dr.cos(th_i) / (n_f * dr.cos(th_i) + n_i * dr.cos(th_f))
    else:
        raise ValueError("Polarization must be 's' or 'p'")

def interface_r(polarization, n_i, n_f, th_i, th_f):
    if polarization == "s":
        return (n_i * dr.cos(th_i) - n_f * dr.cos(th_f)) / (n_i * dr.cos(th_i) + n_f * dr.cos(th_f))
    elif polarization == "p":
        return (n_f * dr.cos(th_i) - n_i * dr.cos(th_f)) / (n_f * dr.cos(th_i) + n_i * dr.cos(th_f))
    else:
        raise ValueError("Polarization must be 's' or 'p'")

def coh_tmm_dr(pol, n_list, d_list, th_0, lam_vac):
    num_layers = dr.width(n_list)

    th_list = list_snell(n_list, th_0)

    kz_list = 2.0 * dr.pi * n_list * dr.cos(th_list) / lam_vac

    delta = kz_list * d_list

    # Get the first and last layer so they can be reset to their original values
    indices = UInt32([0, num_layers - 1])
    delta_result = dr.gather(Complex2f, delta, indices)

    # For a very opaque layer, reset delta to avoid divide-by-0 and similar
    # errors. The criterion imag(delta) > 35 corresponds to single-pass
    # transmission < 1e-30 --- small enough that the exact value doesn't
    # matter.
    delta = dr.select(delta.imag > 35, delta.real + Complex2f(35j), delta)
    # Reset the first and last layer to their original values
    dr.scatter(delta, delta_result, indices)

    # t_list[i,j] and r_list[i,j] are transmission and reflection amplitudes,
    # respectively, coming from i, going to j. Only need to calculate this when
    # j=i+1. (2D array is overkill but helps avoid confusion.)
    t_list = dr.zeros(Complex2f, num_layers * num_layers)
    r_list = dr.zeros(Complex2f, num_layers * num_layers)
    for i in range(num_layers - 1):
        n_list_i = Complex2f(dr.slice(n_list, i))
        n_list_j = Complex2f(dr.slice(n_list, i + 1))
        th_list_i = Complex2f(dr.slice(th_list, i))
        th_list_j = Complex2f(dr.slice(th_list, i + 1))

        t_list_result = interface_t(pol, n_list_i, n_list_j, th_list_i, th_list_j)
        r_list_result = interface_r(pol, n_list_i, n_list_j, th_list_i, th_list_j)

        index = UInt32([i * num_layers + i + 1])
        dr.scatter(t_list, t_list_result, index)
        dr.scatter(r_list, r_list_result, index)


    M_list = dr.zeros(Complex2f, num_layers)

    return { "R" : n_list }

In [ ]:
d_list = Float([dr.inf, 5, 30, dr.inf])
n_list = Complex2f(np.array([1.517, 3.719 + 4.362j, 0.130 + 3.162j, 1]))

lam_vac = 633

theta_list = dr.linspace(Float, dr.deg2rad(30), dr.deg2rad(60), num=300)
Rp = []
for theta in theta_list:
    Rp.append(coh_tmm_dr("p", n_list, d_list, theta, lam_vac)["R"])

In [ ]:
Rp

[[[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.5169999599456787 + 0.0i],
  [3.7190001010894775 + 4.361999988555908i],
  [0.12999999523162842 + 3.1619999408721924i],
  [1.0 + 0.0i]],
 [[1.516999959945678